<a href="https://colab.research.google.com/github/jinhyung426/deeplearning.ai/blob/main/tf_chap3_NLP_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow

## Part 3. Natural Language Processing in TensorFlow

## (2) TFDF Dataset

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tensorflow Dataset

- https://www.tensorflow.org/datasets/catalog/overview
  
  - Image, Video, Text 등 다양한 Dataset 존재

  - ex) IMDB Dataset

- https://github.com/tensorflow/datasets/tree/master/docs/catalog

- Have to enable eager execution (tensorflow 2.x의 경우 default)

In [10]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_data, test_data = imdb['train'], imdb['test']

In [3]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s, l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [4]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, truncating=trunc_type, maxlen=max_length)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [7]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.6490 - accuracy: 0.5932 - val_loss: 0.4399 - val_accuracy: 0.8459
Epoch 2/10
782/782 [==============================] - 4s 6ms/step - loss: 0.3563 - accuracy: 0.8957 - val_loss: 0.4113 - val_accuracy: 0.8340
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2050 - accuracy: 0.9568 - val_loss: 0.4705 - val_accuracy: 0.8213
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1204 - accuracy: 0.9810 - val_loss: 0.4810 - val_accuracy: 0.8277
Epoch 5/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0889 - accuracy: 0.9847 - val_loss: 0.5843 - val_accuracy: 0.8185
Epoch 6/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0706 - accuracy: 0.9873 - val_loss: 0.5738 - val_accuracy: 0.8252
Epoch 7/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0569 - accuracy: 0.9899 - val_loss: 0.6069 - val_accuracy: 0.8248
Epoch 

In [8]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


# Projector

1. **Extract 16 embedded dimensions per word** and save it in a **vecs.tsv** file using io lib
2. Write the** associated word **in a **meta.csv** file
3. Open "projector.tensorflow.org" and project the results
  - load data and choose file
  - Click "Spherize data" to clusterize the results

In [9]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]             # extract 16 embedded dimensions per word
  out_m.write(word + "\n")                   # associated word
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()

In [11]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 68, 105, 12, 7, 491, 1216]]


In [16]:
model.predict(pad_sequences(sequence, maxlen=max_length))

array([[0.97401637]], dtype=float32)